# PydanticOutputParser

PydanticOutputParser는 언어 모델의 출력을 구조화된 정보로 변환하는 데 도움이 되는 클래스로, 언어 모델에서 나온 단순 텍스트를 넘어서, 사용자가 필요로 하는 정보를 명확하고 체계적인 형태로 제공할 수 있음.

## PydanticOutputParser는 주로 두 가지의 핵심 메서드가 구현되어야 함.


*   get_format_instruction: 언어 모델이 출력해야 할 정보의 형식을 정의하는 지침을 제공. 예를 들어, 언어 모델이 출력해야 할 데이터의 필드와 그 형태를 설명하는 지침을 문자열로 반환할 수 있음.
*   parse: 언어 모델의 출력(문자열로 가정)을 받아들여 이를 특정 구조로 분석하고 변환. Pydantic와 같은 도구를 사용하여, 입력된 문자열을 사전 정의된 스키마에 따라 검증하고, 해당 스키마를 따르는 데이터 구조로 변환.



In [ ]:
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.output_parsers import PydanticOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field

llm = ChatOpenAI(temperature=0)

story_content = """
  향단이는 길동이를 다리위에서 칼로 찔러 죽였다.
"""

class Story(BaseModel):
  suspect: str = Field(description="죽인 사람(용의자)")
  victim: str = Field(description="죽은 사람(피해자)")
  point: str = Field(description="장소")
  weapon: str = Field(description="살해 무기")

parser = PydanticOutputParser(pydantic_object=Story)

In [ ]:
prompt = PromptTemplate.from_template(
    """
      아래의 질문에 한국어로 답해주세요.

      질문: {question}
      스토리: {story}
      FORMAT: {format}
    """
)
prompt = prompt.partial(format=parser.get_format_instructions())

In [ ]:
chain = prompt | llm

response = chain.invoke({
    "question": "이 이야기의 요소들을 추출해주세요.",
    "story": story_content
})

print(response.content)

{
  "suspect": "향단이",
  "victim": "길동이",
  "point": "다리위",
  "weapon": "칼"
}


## Parse를 결합한 체인 생성

출력 결과를 객체로 생성할 수 있다.

In [ ]:
chain = prompt | llm | parser

response = chain.invoke({
    "question": "이 이야기의 요소들을 추출해주세요.",
    "story": story_content
})

response

Story(suspect='향단이', victim='길동이', point='다리위', weapon='칼')

# CommaSeparatedListOutputParser

쉼표로 구분된 항목 목록을 반환할 필요가 있을 때 유용하게 사용할 수 있음. 이 출력 파서를 사용하여 사용자가 입력한 데이터나 요청한 정보를 쉼표로 구분하여 명확하고 간결한 목록 형태로 제공받을 수 있음.

In [ ]:
from langchain.output_parsers import CommaSeparatedListOutputParser
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

output_parser = CommaSeparatedListOutputParser()

format_instructions = output_parser.get_format_instructions()
print(format_instructions)

prompt = PromptTemplate(
    template="{subject} 5명.\n{format_instuctions}",
    input_variables=["subject"],
    partial_variables={"format_instuctions": format_instructions}
)

model = ChatOpenAI(temperature=0)

chain = prompt | model | output_parser

Your response should be a list of comma separated values, eg: `foo, bar, baz` or `foo,bar,baz`


In [ ]:
chain.invoke(
    {"subject": "대한민국의 역대 대통령"}
)

['이승만', '박정희', '전두환', '김영삼', '김대중']

## Chain Stream

chain.stream을 사용하여 스트림을 반복처리 가능.

In [ ]:
for p in chain.stream({"subject": "대한민국의 영웅"}):
  print(p)

['유관순']
['안중근']
['이순신']
['김구']
['박근혜']


# StructuredOutputParser

여러 필드를 반환하고자 할 때 유용하게 사용할 수 있음.

*   ResponseSchema: 클래스를 사용하여 응답 스키마를 정의
*   StructuredOutputParser: 응답 스키마들을 사용하여 출력을 구조화



In [ ]:
from langchain.output_parsers import ResponseSchema, StructuredOutputParser
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

In [ ]:
response_schemas = [
    ResponseSchema(name="answer", description="질문의 답변"),
    ResponseSchema(name="source", description="답변의 출처")
]

output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
print(output_parser)

response_schemas=[ResponseSchema(name='answer', description='질문의 답변', type='string'), ResponseSchema(name='source', description='답변의 출처', type='string')]


In [ ]:
format_instructions = output_parser.get_format_instructions()
print(format_instructions)

prompt = PromptTemplate(
    template="질문에 답변해 주세요.\n{format_instructions}\n{question}",
    input_variables=["question"],
    partial_variables={"format_instructions": format_instructions}
)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"answer": string  // 질문의 답변
	"source": string  // 답변의 출처
}
```


In [ ]:
model = ChatOpenAI(temperature=0)
chain = prompt | model | output_parser

In [ ]:
chain.invoke({"question": "미국의 초대 대통령은 누구인가요?"})

{'answer': '존 아담스(John Adams)',
 'source': 'https://en.wikipedia.org/wiki/List_of_Presidents_of_the_United_States'}

## Chain Stream
chain.stream을 사용하여 스트림을 반복처리 가능.

In [ ]:
for s in chain.stream({"question": "맥아더 장군의 업적은 무엇인가요?"}):
  print(s)

{'answer': '맥아더 장군은 한국전쟁 중에 유엔군 사령관으로 활약하여 인천상륙작전과 함께 전쟁의 흐름을 뒤집는 데 큰 역할을 했습니다.', 'source': 'https://ko.wikipedia.org/wiki/%EB%A7%A5%EC%95%84%EB%8D%94_%EC%9E%A5%EA%B5%B0'}
